# ELT Dieese

In [ ]:
%run ../../config/bootstrap.py
from utils import get_project_root,build_row_hash
from pathlib import Path 
#from pandas_scd import scd2
import pandas as pd
from datetime import datetime

In [ ]:

project_root = get_project_root() 
project_root

## ⌛ Staging


In [ ]:
df_stg = pd.read_excel(
    project_root / "data/staging/dieese/dieese_dataset_201001_202504.xls",
    engine="xlrd",
    header=1
)

In [ ]:
df_stg.head()

## 🥉 Bronze

In [ ]:

df_stg = df_stg.rename(columns={df_stg.columns[0]: "data"})  
df_bronze = (
    df_stg
      .melt(id_vars="data",          # mantém a coluna data
            var_name="municipio",    # novos rótulos das colunas “largas”
            value_name="vlr_cesta_basica")  # valores das antigas colunas
      .dropna(subset=["vlr_cesta_basica"])  # remove linhas sem valor
      .reset_index(drop=True)        # limpa o índice
)

# opcional: converte a coluna data para datetime se quiser filtrar por datas depois
df_bronze["data"] = "01-"+df_bronze["data"]

df_bronze.head()

In [ ]:
output_path = project_root / "data" / "bronze" / "dieese" / "dieese.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)
df_bronze.to_csv(output_path, index=False)


In [ ]:
output_path

## 🥈 Silver

In [ ]:
output_path = project_root / "data" / "bronze" / "dieese" / "dieese.csv"

In [ ]:
df_bronze = pd.read_csv(output_path)
df_bronze.head()

In [ ]:
df_bronze.municipio.unique()

In [ ]:

city_to_ibge = {
    "Brasília":      5300108,
    "Campo Grande":  5002704,
    "Cuiabá":        5103403,
    "Goiânia":       5208707,
    "Belo Horizonte":3106200,
    "Rio de Janeiro":3304557,
    "São Paulo":     3550308,
    "Vitória":       3205309,
    "Curitiba":      4106902,
    "Florianópolis": 4205407,
    "Porto Alegre":  4314902,
    "Belém":         1501402,
    "Boa Vista":     1400100,
    "Macapá":        1600303,
    "Manaus":        1302603,
    "Palmas":        1721000,
    "Porto Velho":   1100205,
    "Rio Branco":    1200401,
    "Aracaju":       2800308,
    "Fortaleza":     2304400,
    "João Pessoa":   2507507,
    "Maceió":        2704302,
    "Natal":         2408102,
    "Recife":        2611606,
    "Salvador":      2927408,
    "São Luís":      2111300,
    "Teresina":      2211001,
    "Macaé":         3302403
}

In [ ]:
df_silver = df_bronze.copy()
df_silver["id_municipio"] = df_bronze["municipio"].map(city_to_ibge)
df_silver = df_silver[['data','municipio','id_municipio','vlr_cesta_basica']]
df_silver.head()

### Quality

In [ ]:
missing = df_silver.loc[df_silver["id_municipio"].isna(), "municipio"].unique()
if len(missing):
    print("Cidades sem código encontrado:", missing)


### hash

In [ ]:
df_silver.columns

In [ ]:
cols_for_hash = ['data', 'municipio', 'id_municipio', 'vlr_cesta_basica']

In [ ]:
df_silver["hash"] = build_row_hash(df_silver, cols_for_hash, algo="sha256")
df_silver.info()

In [ ]:
df_silver.hash.nunique()

### scd2

In [ ]:


# tgt = dimensão existente (pode estar vazia)
tgt = pd.DataFrame({
    "municipio": [4314902],
    "populacao": [1488252],
    "start_": [datetime(2024, 1, 1)],
    "end_": [None],
    "is_active": [True]
})

# src = carga nova
src = pd.DataFrame({
    "municipio": [4314902],
    "populacao": [1491000]   # mudou!
})

final_dim = scd2(src, tgt, cols_to_track=["populacao"])


### save

In [ ]:
output_path = project_root / "data" / "silver" / "dieese" / "dieese.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)
df_silver.to_csv(output_path, index=False)


## 🥇 Gold

In [ ]:
silver_path = project_root / "data" / "silver" / "dieese" / "dieese.csv"
gold_path   = project_root / "data" / "gold"  / "dieese" / "dieese.csv"


gold_path.parent.mkdir(parents=True, exist_ok=True)

# df_silver = pd.read_csv(silver_path, parse_dates=["start_date", "end_date"])

# --- filtra registros vigentes ---
df_gold = df_silver.loc[df_silver["is_current"] == 1].copy()

# --- grava a gold ---
df_gold.to_csv(gold_path, index=False)

print(f"Salvo {len(df_gold):,} registros atuais em {gold_path}")
